In [0]:
import os
from PIL import Image
import numpy as np
from IPython.display import display
import h5py 
import cv2

In [0]:
from __future__ import print_function
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten , BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [0]:
# parameters for data
height = 224
width = 224
channels = 3
input_shape = (height, width, channels)
n_classes = 100
# Parameters for training
epochs = 15
batch_size = 32

In [0]:
from keras.datasets import cifar100
(X, y), (X_test, y_test) = cifar100.load_data()

In [0]:
from __future__ import print_function
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten , BatchNormalization
from keras.layers import Conv2D, MaxPooling2D,MaxPooling3D
from keras import backend as K

In [0]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

import keras
import keras.backend as K
from keras.datasets import cifar100
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Flatten, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical

import albumentations as albu
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score

In [0]:
# Spliting the training data into a training data and a validation data.
st = StratifiedShuffleSplit(n_splits = 2, test_size = 0.2, random_state = 1)
for train_index, val_index in st.split(X, y):
    X_train, X_val, y_train, y_val = X[train_index], X[val_index], y[train_index], y[val_index]
    
print("The number of training data : ", X_train.shape[0])
print("The number of validation data : ", X_val.shape[0])

del X, y

The number of training data :  40000
The number of validation data :  10000


In [0]:
def np_resize(img, shape):
    return cv2.resize(img, (shape[1], shape[0]), interpolation = cv2.INTER_CUBIC)

In [0]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for keras'
    def __init__(self, images , labels = None, mode = 'fit', batch_size = batch_size,
                 dim = (height, width), channels = channels, n_classes = n_classes,
                 shuffle = True, augment = False):
        self.images = images
        self.labels = labels
        self.mode = mode
        self.batch_size = batch_size
        self.dim = dim
        self.channels = channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.augment = augment
        
        self.on_epoch_end()
        
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(self.images.shape[0])
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
            
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.images) / self.batch_size))
        
    def __getitem__(self, index):
        'Generate one batch of data'
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # =========================================================== #
        # Generate mini-batch of X
        # =========================================================== #
        X = np.empty((self.batch_size, *self.dim, self.channels))
        for i, ID in enumerate(batch_indexes):
            # Generate a preprocessed image
            img = self.images[ID]
            img = img.astype(np.float32) / 255.
            img = np_resize(img, self.dim)
            X[i] = img
            
        
        # =========================================================== #
        # Generate mini-batch of y
        # =========================================================== #
        if self.mode == 'fit':
            y = self.labels[batch_indexes]
            y = to_categorical(y, n_classes)
            '''
            y = np.zeros((self.batch_size, self.n_classes), dtype = np.uint8)
            for i, ID in enumerate(batch_indexes):
                # one hot encoded label
                y[i, self.labels[ID]] = 1
            '''
            # Augmentation should only be implemented in the training part.
            if self.augment == True:
                X = self.__augment_batch(X)                
            
            return X,y
        
        elif self.mode == 'predict':
            return X       
        
        else:
            raise AttributeError('The mode parameters should be set to "fit" or "predict"')
            
    def __random_transform(self, img):
        composition = albu.Compose([albu.HorizontalFlip(p = 0.5),
                                    albu.VerticalFlip(p = 0.5),
                                    albu.GridDistortion(p = 0.2),
                                    albu.ElasticTransform(p = 0.2)])
        
        return composition(image = img)['image']
        
    
    def __augment_batch(self, img_batch):
        for i in range(img_batch.shape[0]):
            img_batch[i] = self.__random_transform(img_batch[i])
            
        return img_batch

In [0]:
train_generator = DataGenerator(X_train, y_train, augment = True)
valid_generator = DataGenerator(X_val, y_val, augment = False)

In [0]:
import os
print(os.getcwd())

/content/drive/My Drive


In [0]:
import numpy as np
weights_dic = np.load('/content/bvlc_alexnet.npy', encoding='bytes',allow_pickle=True).item()

In [0]:
model = Sequential()
# 1st layer..........
model.add(Conv2D(96, kernel_size = (11, 11), strides = (4,4), activation='relu', padding = 'valid' , data_format='channels_last', input_shape=(227, 227, 3),name = "first"))
model.add(MaxPooling2D(pool_size=(3,3) , strides = (2,2) , padding = 'valid' )) 
model.add(BatchNormalization())

# 2nd layer................
model.add(Conv2D(256, kernel_size=(5,5), activation='relu', strides=(1,1), padding = 'valid', name = "second"))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
model.add(BatchNormalization())

#3rd layer.................
model.add(Conv2D(384, kernel_size=(3,3), activation='relu', strides=(1,1), padding='valid', name = "third"))
model.add(BatchNormalization())

#4th layer.................
model.add(Conv2D(384, kernel_size=(3,3), activation='relu', strides=(1,1), padding='valid', name = "fourth"))
model.add(BatchNormalization())

#5th layer.................
model.add(Conv2D(256, kernel_size=(3,3), strides=(1,1), activation = 'relu', padding='valid' , name = "fifth"))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
model.add(BatchNormalization())

model.add(Dropout(0.2))

model.add(Flatten())

# Fully Connected Layer 1
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.4))

# Fully Connected Layer 2
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(n_classes, activation = 'softmax')) #As number of nodes in last layer in softmax is number of classes, where each node is probability of classes

model.compile(loss=keras.losses.categorical_crossentropy,
          optimizer=keras.optimizers.Adadelta(),
          metrics=['accuracy'])













Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
print(model.get_layer("second").get_weights()[0].shape)

(5, 5, 96, 256)


In [0]:
model.get_layer("first").set_weights([weights_dic["conv1"][0], weights_dic["conv1"][1]])
# model.get_layer("second").set_weights([weights_dic["conv2"][0], weights_dic["conv2"][1]])
# model.get_layer("third").set_weights([weights_dic["conv3"][0], weights_dic["conv3"][1]])
# model.get_layer("fourth").set_weights([weights_dic["conv4"][0], weights_dic["conv4"][1]])
# model.get_layer("fifth").set_weights([weights_dic["conv5"][0], weights_dic["conv5"][1]])

In [0]:
from keras.models import load_model
model = load_model("model.h5")

In [0]:
from time import time
tensorboard = keras.callbacks.TensorBoard(log_dir="pretrainedLogs/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)

In [0]:
model.fit_generator(train_generator,validation_data = valid_generator, 
                           epochs = epochs, verbose = 1,callbacks=[tensorboard])




Epoch 1/15
1250/1250 [==============================] - 518s 415ms/step - loss: 2.2089 - acc: 0.4402 - val_loss: 2.1739 - val_acc: 0.4475

Epoch 2/15
1250/1250 [==============================] - 530s 424ms/step - loss: 2.1919 - acc: 0.4435 - val_loss: 2.1260 - val_acc: 0.4551
Epoch 3/15
1250/1250 [==============================] - 526s 421ms/step - loss: 2.1713 - acc: 0.4470 - val_loss: 2.2007 - val_acc: 0.4415
Epoch 4/15
1250/1250 [==============================] - 517s 413ms/step - loss: 2.1813 - acc: 0.4490 - val_loss: 2.0748 - val_acc: 0.4668
Epoch 5/15
1250/1250 [==============================] - 522s 418ms/step - loss: 2.1628 - acc: 0.4531 - val_loss: 2.0896 - val_acc: 0.4695
Epoch 6/15
1250/1250 [==============================] - 523s 418ms/step - loss: 2.1468 - acc: 0.4583 - val_loss: 2.0858 - val_acc: 0.4724
Epoch 7/15
1250/1250 [==============================] - 531s 424ms/step - loss: 2.1431 - acc: 0.4587 - val_loss: 2.1771 - val_acc: 0.4551
Epoch 8/15
1250/1250 [=========

In [0]:
model.save("pretrainedModel1.h5")

In [0]:
# Make sure to set shuffle be False.
test_generator = DataGenerator(X_test, mode = 'predict', augment = False, shuffle = False)
y_pred = model.predict_generator(test_generator,verbose = 1)
y_pred = np.argmax(y_pred, axis = 1)

312/312 [==============================] - 7s 22ms/step


In [0]:
print(y_pred)
y_test = y_test[0:9984]
print(y_test)

[49  4 55 ... 59 21 42]
[[49]
 [33]
 [72]
 ...
 [52]
 [74]
 [29]]


In [0]:
print("Testing Accuracy: ",accuracy_score(y_test, y_pred) * 100, "%")

Testing Accuracy:  49.05849358974359 %
